In [2]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

def readConfigFile(file):
    with open(file,'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as error:
            return logging.error(error)

def replacer(string, char):
    pattern = char +'{2,}'
    string = re.sub(pattern,char, string)
    return string

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex = True)
    df.columns = list(map(lambda x: x.strip('_'),list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'),list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),list(table_config['columns'])))
    expected_col.sort()
    df.columns = list(map(lambda x: x.lower(),list(df.columns)))
    df = df.reindex(sorted(df.columns),axis=1)

    if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
        print('Col names and col len validation passed')
        return 1
    else:
        print('Col names and col len validation failed')
        mismatched_col_file = list(set(df.columns).difference(expected_col))
        print('The following cols are not in the YAML File',mismatched_col_file)
        missing_YAML = list(set(expected_col).difference(df.columns))
        print('Following YAML cols are not in the file',missing_YAML)
        logging.info(f'df.columns:{df.columns}')
        logging.info(f'expected columns:{expected_col}')
        return 0


Writing testutility.py


In [1]:
%%writefile file.yaml
file_type: csv
dataset_name: washingtonViolations
file_name: WA
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
    - id
    - stop_date
    - location_raw
    - driver_gender
    - driver_age
    - violation
    - stop_outcome
 


Overwriting file.yaml


In [2]:
import testutility as util
config_data = util.readConfigFile('file.yaml')

In [3]:
config_data
#dict

{'file_type': 'csv',
 'dataset_name': 'washingtonViolations',
 'file_name': 'WA',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['id',
  'stop_date',
  'location_raw',
  'driver_gender',
  'driver_age',
  'violation',
  'stop_outcome']}

In [4]:
#read file using YAML preset
import pandas as pd
file_type = config_data['file_type']
source_file = './'+config_data['file_name'] + f'.{file_type}'
df = pd.read_csv(source_file,config_data['inbound_delimiter'])

c:\Users\antho\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [1]:
util.col_header_val(df,config_data)
 

NameError: name 'util' is not defined

In [8]:
f = open('summary.txt','w')
f.write('File Size: '+ str(os.path.getsize(source_file) * 10 ** -9) + ' GB' + '\n' + 'Total # of Rows: ' + str(len(pd.read_csv(source_file,config_data['inbound_delimiter']).index)) + '\n' + 'Total # of Columns: ' +  str(len(pd.read_csv(source_file,config_data['inbound_delimiter']).columns)))

f.close()

c:\Users\antho\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
import csv
reader = csv.reader(open(source_file, newline=None), delimiter=config_data['inbound_delimiter'])
writer = csv.writer(open('output.csv', 'w'), delimiter=config_data['outbound_delimiter'])
writer.writerows(reader)